# Transformaciones 2D con Matrices y GIF

Este notebook:

- Crea una figura 2D (un cuadrado).
- Aplica traslacion, rotacion y escala con matrices homogeneas 3x3.
- Anima la transformacion en funcion del frame `t`.
- Exporta el resultado como GIF animado usando `imageio`.
- (Opcional) Muestra matrices de transformacion en distintos frames.

In [ ]:
# En Colab normalmente imageio ya existe; descomenta si hace falta.
# %pip install imageio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio

from pathlib import Path
from IPython.display import Image, display

In [ ]:
def T(tx, ty):
    return np.array([
        [1.0, 0.0, tx],
        [0.0, 1.0, ty],
        [0.0, 0.0, 1.0],
    ])

def R(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [c, -s, 0.0],
        [s,  c, 0.0],
        [0.0, 0.0, 1.0],
    ])

def S(sx, sy):
    return np.array([
        [sx, 0.0, 0.0],
        [0.0, sy, 0.0],
        [0.0, 0.0, 1.0],
    ])

In [ ]:
# Figura base (cuadrado) en coordenadas homogeneas: columnas = puntos
shape = np.array([
    [-0.5,  0.5,  0.5, -0.5, -0.5],
    [-0.5, -0.5,  0.5,  0.5, -0.5],
    [ 1.0,  1.0,  1.0,  1.0,  1.0],
])

n_frames = 90
gif_path = Path('transformaciones_2d.gif')
frames = []
matrices_por_frame = []

fig, ax = plt.subplots(figsize=(6, 6))

for i in range(n_frames):
    t = i / (n_frames - 1)  # t en [0, 1]

    # Parametros que cambian con el tiempo
    theta = 2 * np.pi * t
    tx = -2.0 + 4.0 * t
    ty = 0.8 * np.sin(2 * np.pi * t)
    sx = 1.0 + 0.5 * np.sin(2 * np.pi * t)
    sy = 1.0 + 0.5 * np.cos(2 * np.pi * t)

    M_t = T(tx, ty)
    M_r = R(theta)
    M_s = S(sx, sy)

    # Orden: primero escala, luego rota, luego traslada
    M = M_t @ M_r @ M_s
    matrices_por_frame.append((t, M_t, M_r, M_s, M))

    transformed = M @ shape

    ax.clear()
    ax.plot(shape[0], shape[1], 'k--', alpha=0.45, label='Original')
    ax.fill(transformed[0], transformed[1], color='#1f77b4', alpha=0.70, label='Transformada')
    ax.plot(transformed[0], transformed[1], color='#0d3c67')
    ax.set_xlim(-3.2, 3.2)
    ax.set_ylim(-3.2, 3.2)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, alpha=0.25)
    ax.legend(loc='upper right')
    ax.set_title(f'Frame {i:02d} | t={t:.2f}')

    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frames.append(frame.copy())

plt.close(fig)

imageio.mimsave(gif_path, frames, duration=0.055, loop=0)
display(Image(filename=str(gif_path)))
print(f'GIF guardado en: {gif_path.resolve()}')

In [ ]:
# Opcional: inspeccionar matrices en algunos frames
indices = [0, n_frames // 3, 2 * n_frames // 3, n_frames - 1]

for idx in indices:
    t, M_t, M_r, M_s, M = matrices_por_frame[idx]
    print('=' * 72)
    print(f'Frame {idx} | t={t:.3f}')
    print('M_traslacion =\n', np.round(M_t, 3))
    print('M_rotacion =\n', np.round(M_r, 3))
    print('M_escala =\n', np.round(M_s, 3))
    print('M_total = M_t @ M_r @ M_s =\n', np.round(M, 3))